In [1]:
import requests
import pandas as pd
import sqlite3

server = "https://www.ages.at/typo3temp/badegewaesser_db.json"


In [2]:
response = requests.get(server)
details = response.json()
data = []
for item in details["BUNDESLAENDER"]: 
    data = data + item['BADEGEWAESSER']

for item in data:     
    name = item["BADEGEWAESSERNAME"]
    values = item["MESSWERTE"][0]
    E_COLI, Int_Ent, Temp = values["E"], values["E_C"], values["W"]

    print(f"{name}: \nE.Coli: {E_COLI} KBE/100ml\nIntestinal Enterococci: {Int_Ent} KBE/100ml\nTemperature: {Temp}°C\n")


Naturbadesee Königsdorf: 
E.Coli: 21 KBE/100ml
Intestinal Enterococci: 15 KBE/100ml
Temperature: 26.2°C

Sonnensee Ritzing: 
E.Coli: 15 KBE/100ml
Intestinal Enterococci: 15 KBE/100ml
Temperature: 25.3°C

Neusiedlersee, Rust: 
E.Coli: 15 KBE/100ml
Intestinal Enterococci: 15 KBE/100ml
Temperature: 26.3°C

Neusiedlersee, Breitenbrunn: 
E.Coli: 110 KBE/100ml
Intestinal Enterococci: 332 KBE/100ml
Temperature: 26.2°C

Neusiedlersee, Mörbisch: 
E.Coli: 61 KBE/100ml
Intestinal Enterococci: 15 KBE/100ml
Temperature: 26.3°C

Neufelder See: 
E.Coli: 15 KBE/100ml
Intestinal Enterococci: 46 KBE/100ml
Temperature: 25.7°C

Steinbrunner See: 
E.Coli: 30 KBE/100ml
Intestinal Enterococci: 15 KBE/100ml
Temperature: 26.3°C

Stausee Forchtenstein: 
E.Coli: 15 KBE/100ml
Intestinal Enterococci: 15 KBE/100ml
Temperature: 23.3°C

Badesee Neudörfl: 
E.Coli: 15 KBE/100ml
Intestinal Enterococci: 15 KBE/100ml
Temperature: 24.4°C

Römersee Wiesen: 
E.Coli: 15 KBE/100ml
Intestinal Enterococci: 15 KBE/100ml
Temperatu

In [3]:
df = pd.json_normalize(data)

In [4]:
locations = df[[
    "BADEGEWAESSERID", "BADEGEWAESSERNAME", "BEZIRK", "LATITUDE", "LONGITUDE"
]].rename(columns={
    "BADEGEWAESSERID": "id",
    "BADEGEWAESSERNAME": "name",
    "BEZIRK": "alternate_name",
    "LATITUDE": "lat",
    "LONGITUDE": "long"
}).set_index("id")
locations

,name,alternate_name,lat,long
id,,,,
AT1051051100150010,Naturbadesee Königsdorf,Jennersdorf,47.0082870000,16.1632530000
AT1110002000120010,Sonnensee Ritzing,Oberpullendorf,47.629901,16.470699
AT1120000200060010,"Neusiedlersee, Rust",Rust(Stadt),47.8026,16.700199
AT1120000300060010,"Neusiedlersee, Breitenbrunn",Eisenstadt-Umgebung,47.914798,16.766899
AT1120000900060010,"Neusiedlersee, Mörbisch",Eisenstadt-Umgebung,47.752799,16.6993
...,...,...,...,...
AT3420002300060010,"Rheinauen, Hohenems",Dornbirn,47.367698,9.654709
AT3420002400010010,"Alter Rhein, Lustenau",Dornbirn,47.399299,9.65954
AT3420002900080010,"Untere Au, Frastanz",Feldkirch,47.2233,9.63467


In [5]:
db = sqlite3.connect("../dataset.sqlite3")
locations.to_sql("locations", db, if_exists="append")


258